<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-2-Opti.-Mat./blob/main/P2_Tarea_2_Opti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install gurobipy

In [45]:
import numpy as np
from gurobipy import*

In [46]:
#mejor utilizamos librerias para resolver el problema xd:

betha = {
    0:0,
    1:100,
    2:120,
    3:130,
    4:140,
    5:150,
    6:200,
    7:0
} #betha es un vector columna


c = {
    0:{1:10, 2:25, 3:30, 6:20},
    1:{4:15, 5:15},
    2:{5:15, 6:0},
    3:{6:10},
    4:{7:25},
    5:{7:80},
    6:{7:25},
    7:{}
}  # c es una matriz de 8x8


r = {
    0: {1: 500, 2: 350, 3: 500, 6: 600},
    1: {4: 250, 5: 350},
    2: {5: 350},
    3: {6: 500},
    4: {7: 800},
    5: {7: 500},
    6: {7: 500},
    7: {}
}  # r es una matriz de 8x8

n = 8


m = Model('Penalización')

#defino la varaible


x = m.addVars(n,n, vtype= GRB.CONTINUOUS,lb= 0, name = 'elemtos_por_bodega')

#añado las restricciones:


#hay que bloquear aquellos caminos donde r[i][j] == 0
for i in range(n):
  for j in range(n):
    if j not in r[i]:
      m.addConstr(x[i,j] == 0, name=f'Bloqueo_{i}_{j}')
    else:
      m.addConstr(x[i,j] <= r[i][j], name=f'Capacidad_{i}_{j}')


#Primera bodega tenga 1000

m.addConstr(quicksum(x[0, j] for j in range(n)) == 1000, name='Inicio')

#Ultima bodega tenga 1000

m.addConstr(quicksum(x[i,7] for i in range(n)) == 1000, name='Llegada')


#lo que entra por un nodo debe ser igual a lo que sale por este:

for i in range(1,7):
  entrada = quicksum(x[j,i] for j in range(n))
  salida = quicksum(x[i,j] for j in range(n))
  m.addConstr(entrada == salida, name=f'balance_{i}')

#definimos la función Objetivo:

costo_transporte = quicksum(c[i][j]*(x[i,j]*x[i,j]) for i in range(n) for j in range(n) if j in r[i])

costo_bodegaje = quicksum(betha[j]*(quicksum(x[k,j] for k in range(n))) for j in range(n) if j)

m.setObjective(costo_bodegaje + costo_transporte, GRB.MINIMIZE)

m.optimize()

if m.status == GRB.OPTIMAL:
    print("Solución óptima encontrada:")
    for i in range(n):
        for j in range(n):
            if x[i, j].x > 1e-6:  # Evitar imprimir valores cercanos a cero
                print(f'x[{i},{j}] = {x[i, j].x:.2f}')
else:
    print("No se encontró solución óptima")




Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 72 rows, 64 columns and 164 nonzeros
Model fingerprint: 0xedd26eba
Model has 11 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  QObjective range [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 65 rows and 53 columns
Presolve time: 0.02s
Presolved: 7 rows, 11 columns, 18 nonzeros
Presolved model has 11 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.000e+00
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.547854

In [62]:
#ahora tenemos que tratar de lograr hacer la p2. mi unico problema es que necesito tener solo igualdades como estriccion.

#DEFINICIOÓN DE CONSTANTES PARA ITERACIÓN.

mu = 1

for k in range(10):
  epsilon = 10**-5
b = 10

m2 = Model('Penalización')

x = m2.addVars(n,n, vtype= GRB.CONTINUOUS,lb= 0, name = 'elemtos_por_bodega')

R0 = quicksum(x[0, j] for j in range(n)) - 1000

R1 = quicksum(x[i,7] for i in range(n)) - 1000

R2 = []  # Balance de flujo para nodos intermedios

for k in range(1, 7):
    R2.append(quicksum(x[i, k] for i in range(n)) - quicksum(x[k, j] for j in range(n)))

#Esto de aca define las cotas inferiores y superiores Q en las cuales esta definido x

for i in range(n):
    for j in range(n):
        if j in r[i]:  # Asegurar que la ruta existe
            m2.addConstr(x[i,j] <= r[i][j], f"Cota_superior_{i}_{j}")
        else:
          m2.addConstr(x[i,j] == 0, f"No_negatividad_{i}_{j}")

#entonces ahora debemos definir phi

costo_transporte = quicksum(c[i][j] * x[i,j] * x[i,j] for i in range(n) for j in c[i])
costo_bodegaje = quicksum(betha[j] * quicksum(x[i,j] for i in range(n)) for j in range(n))

f = costo_transporte + costo_bodegaje

penalizacion = R0 * R0 + R1 * R1 + quicksum(r * r for r in R2)

m2.setObjective(f + mu * penalizacion, GRB.MINIMIZE)

# Resolver

m2.optimize()

if m2.status == GRB.OPTIMAL:
    #imprimir el resultado que obtuvimos y evaluarlo
    R2_value = [r.getValue() for r in R2]
    valor_penalizacion = (R0.getValue()**2 + R1.getValue()**2 + sum(r**2 for r in R2_value))
    print(f"Iteración {k+1}: mu={mu}, penalización={mu*valor_penalizacion:.4f}, f(x) = {f.getValue()}, Phi(x) = {f.getValue()+mu*valor_penalizacion}")





Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 64 rows, 64 columns and 64 nonzeros
Model fingerprint: 0x4dbfb4ba
Model has 644 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+03]
  QObjective range [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve removed 64 rows and 55 columns
Presolve time: 0.01s
Presolved: 0 rows, 9 columns, 0 nonzeros
Presolved model has 22 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 8
 AA' NZ     : 2.300e+01
 Factor NZ  : 3.600e+01
 Factor Ops : 2.040e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
 